In [1]:
import numpy as np
import pandas as pd
from os.path import join
import os
from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib qt

import nilmtk
from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.disaggregate import CombinatorialOptimisation, FHMM
from nilmtk.utils import print_dict
from nilmtk.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days


In [2]:
import sys

In [3]:
!cp ../../code/common/

usage: cp [-R [-H | -L | -P]] [-fi | -n] [-apvX] source_file target_file
       cp [-R [-H | -L | -P]] [-fi | -n] [-apvX] source_file ... target_directory


In [3]:
sys.path.append("../../code/common/")
sys.path.append("../../code/fridge/")

In [4]:
ds = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")
fridges = nilmtk.global_meter_group.select_using_appliances(type='fridge')

In [5]:
fridges_id_building_id = {i:fridges.meters[i].building() for i in range(len(fridges.meters))}

In [6]:
fridge_id_building_id_ser = pd.Series(fridges_id_building_id)

In [43]:
fridge_id_building_id_ser.describe()

count    174.000000
mean     116.724138
std       69.976245
min        1.000000
25%       54.500000
50%      116.500000
75%      175.750000
max      238.000000
dtype: float64

In [10]:
from fridge_compressor_durations_optimised_jul_7 import compressor_powers, defrost_power

In [11]:
fridge_ids_to_consider = compressor_powers.keys()

In [45]:
len(fridge_ids_to_consider)

96

In [12]:
Wm_to_kwh = 1.66666667 * 1e-5

def wm_to_kwh_per_month(wm, mins):
    return wm * Wm_to_kwh / (mins * 1.0 / (1440 * 30))

In [13]:
building_ids_to_consider = fridge_id_building_id_ser[fridge_ids_to_consider]

In [14]:

building_ids_to_consider.head()

1       1
2      10
8     105
11     11
13    112
dtype: int64

In [76]:
import glob

In [79]:
list_of_files = glob.glob("output/*.h5")

In [80]:
len(list_of_files)

93

In [81]:
out = {}
for f in list_of_files:
    print f.split("/")[1].split(".")[0]
    df = pd.HDFStore(f)['/disag']
    df_energy = wm_to_kwh_per_month(df.sum(), len(df))
    out[f.split("/")[1].split(".")[0]] = df_energy.to_dict()
    #out[f.split("/")[1].split(".")[0]] = df.sum()

1
100
102
103
104
107
109
11
110
112
114
115
116
118
119
123
124
125
126
128
129
13
130
131
133
134
135
136
138
139
14
140
142
144
145
146
149
15
151
152
153
154
155
157
158
159
161
163
167
169
18
2
22
25
29
33
34
35
37
42
43
44
45
46
47
50
51
52
54
55
57
58
59
60
61
67
68
72
75
76
78
79
8
83
84
87
88
89
92
93
95
97
99


In [22]:
from common_functions import latexify, format_axes
latexify()
ax = pd.DataFrame(out).T.dropna().plot(kind="bar")
plt.xlabel("Home")
plt.ylabel("Monthly fridge energy (kWh)")
format_axes(ax)
plt.tight_layout()
plt.savefig("../../figures/fridge/disag_algos_energy.pdf", bbox_inches="tight")
plt.savefig("../../figures/fridge/disag_algos_energy.png", bbox_inches="tight")

In [19]:
from sklearn import linear_model
import scipy

In [62]:
d1 = pd.DataFrame(out).T

In [65]:
d1.isnull().sum()

CO      2
FHMM    1
GT      0
Hart    3
dtype: int64

In [82]:
d = pd.DataFrame(out).T.dropna()

In [83]:
import math
def sign(n):
    if n>0.0:
        return "+"
    else:
        return "-"

In [84]:
d[["GT"]].describe()

,GT
count,90.000000
mean,60.391810
std,22.994907
min,17.514481
25%,43.720865
50%,54.087402
75%,72.963909
max,159.131240


In [85]:
np.random.seed(42)
latexify(columns=2, fig_height=2.6)
fig, ax = plt.subplots(ncols=3, sharey=True)
for i, algo in enumerate(["CO", "FHMM", "Hart"]):
    x = d[["GT"]]
    y = d[[algo]]
   

    # Train the model using the training sets
    
    model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
    model_ransac.fit(x, y)
    ax[i].scatter(x, y, color="gray", alpha=0.5)
    
    ax[i].set_xlabel("Actual energy (kWh)")
    
    ax[i].plot(x, model_ransac.predict(x), color='black')
    format_axes(ax[i])
    mean_average_error = np.mean(np.abs(y-model_ransac.predict(x))/y)*100
    title = algo + "\n" + "y = %0.2f x %s %0.2f \n Energy Error= %0.1f percent" %(model_ransac.estimator_.coef_[0], 
                                                   sign(model_ransac.estimator_.intercept_[0]),
                                                   math.fabs(model_ransac.estimator_.intercept_[0]),
                                                                   mean_average_error)
    ax[i].set_title(title)
ax[0].set_ylabel("Predicted energy (kWh)")
plt.tight_layout()
plt.savefig("../../figures/fridge/disag_algos_energy_scatter.pdf", bbox_inches="tight")
plt.savefig("../../figures/fridge/disag_algos_energy_scatter.png", bbox_inches="tight")

In [31]:
out_energy = {}
for algo in ["CO", "FHMM", "Hart"]:
    out_energy[algo]=(d["GT"]-d[algo]).abs().div(d["GT"])
    

In [33]:
pd.DataFrame(out_energy).plot(kind="box")

In [34]:
pd.DataFrame(out_energy).describe()

,CO,FHMM,Hart
count,52.000000,52.000000,52.000000
mean,0.938419,0.399077,0.317829
std,1.052879,0.537793,0.271267
min,0.092536,0.003751,0.012765
25%,0.288208,0.106370,0.121365
50%,0.459776,0.219939,0.235434
75%,1.229137,0.423629,0.427221
max,6.083893,2.870116,1.214972


In [35]:
d.head()

,CO,FHMM,GT,Hart
1,105.626220,99.891341,25.810942,38.951076
11,101.907341,52.027545,72.995050,55.851776
13,68.663017,67.137810,60.774502,45.489593
139,67.847249,52.004537,53.040532,20.788166
14,158.378926,111.074274,53.151874,47.456254


In [51]:
from sklearn.metrics import f1_score
f_score = {}
limit=20
for f in list_of_files:
    try:
        df = pd.HDFStore(f)['/disag']
        gt = (df>20)[["GT"]]
        pred = (df>20)[["CO", "FHMM", "Hart"]]
        o={}
        for algo in ["CO", "FHMM", "Hart"]:
            o[algo] = f1_score(gt["GT"], pred[algo] )
        f_score[f.split("/")[1].split(".")[0]] = pd.Series(o)
    except:
        pass

In [49]:
gt["GT"].head()

localminute
2014-05-01 00:00:00-05:00    True
2014-05-01 00:01:00-05:00    True
2014-05-01 00:02:00-05:00    True
2014-05-01 00:03:00-05:00    True
2014-05-01 00:04:00-05:00    True
Freq: 60S, Name: GT, dtype: bool

In [50]:
pred[algo].head()

localminute
2014-05-01 00:00:00-05:00    False
2014-05-01 00:01:00-05:00    False
2014-05-01 00:02:00-05:00    False
2014-05-01 00:03:00-05:00    False
2014-05-01 00:04:00-05:00    False
Freq: 60S, Name: Hart, dtype: bool

In [55]:
pd.DataFrame(f_score).T.describe()

,CO,FHMM,Hart
count,52.000000,52.000000,52.000000
mean,0.625507,0.606242,0.662307
std,0.110851,0.137912,0.157258
min,0.367521,0.337143,0.250382
25%,0.557075,0.508627,0.588966
50%,0.621910,0.603452,0.676076
75%,0.684253,0.683499,0.781527
max,0.979902,0.999992,0.893847


In [111]:
gt = (df>20)[["GT"]]
gt.head()

,GT
localminute,
2014-03-17 00:00:00-05:00,True
2014-03-17 00:01:00-05:00,True
2014-03-17 00:02:00-05:00,False
2014-03-17 00:03:00-05:00,False
2014-03-17 00:04:00-05:00,False


In [112]:
algo = "Hart"
x = (df>20)[[algo]]
x.head()

,Hart
localminute,
2014-03-17 00:00:00-05:00,False
2014-03-17 00:01:00-05:00,False
2014-03-17 00:02:00-05:00,False
2014-03-17 00:03:00-05:00,False
2014-03-17 00:04:00-05:00,False


In [118]:
from sklearn.metrics import f1_score

In [119]:
f1_score(gt["GT"], x["Hart"] )

0.76383016455259056

In [117]:
(x["Hart"]==gt["GT"]).head().sum()

3

In [15]:
for f_id, b_id in building_ids_to_consider.head(3).iteritems():
    out[f_id] = {}
    elec = ds.buildings[b_id].elec
    mains = elec.mains()
    elec.appliances
    fridge_instance = fridges.meters[f_id].appliances[0].identifier.instance
    # Dividing train, test
    train_fraction = 0.5
    train = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")
    test = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")
    split_point = elec.train_test_split(train_fraction=train_fraction).date()
    train.set_window(end=split_point)
    test.set_window(start=split_point)
    train_elec = train.buildings[b_id].elec
    test_elec = test.buildings[b_id].elec
    test_mains = test_elec.mains()
    
    # GT elec
    gt_fridge  = test_elec[('fridge', fridge_instance)]
    
    # Pred elec
    for clf_name in cls_dict.keys():
        disag_filename = '%s/%d.h5' %(clf_name, f_id)
        ds_pred = DataSet(disag_filename)
        out[f_id][clf_name] = {}
        pred_fridge = ds_pred.buildings[b_id].elec[('fridge', fridge_instance)]
        out[f_id][clf_name]["pred_energy"] = pred_fridge.total_energy()['active']
        out[f_id][clf_name]["gt_energy"] = gt_fridge.total_energy()['active']
    
    
    
   
    


In [16]:
out

{1: {'CO': {'gt_energy': 25.901466666666668,
   'pred_energy': 268.47235000000001},
  'FHMM': {'gt_energy': 25.901466666666668,
   'pred_energy': 197.42403333333334}},
 2: {'CO': {'gt_energy': 125.88225, 'pred_energy': 175.04763333333332},
  'FHMM': {'gt_energy': 125.88225, 'pred_energy': 154.08428333333333}},
 8: {'CO': {'gt_energy': 154.89259999999999, 'pred_energy': 374.2894},
  'FHMM': {'gt_energy': 154.89259999999999,
   'pred_energy': 261.03871666666669}}}

In [17]:
disag_filename
disag = DataSet(disag_filename)
disag_elec = disag.buildings[b_id].elec

In [18]:
disag_elec.plot()


In [52]:
fridge_elec in top_k_train_elec.meters

False

In [54]:
fridge_elec not in top_k_train_elec.meters

True

In [8]:
building_number = 11
fridge_id = 2

In [9]:
elec = ds.buildings[building_number].elec
mains = elec.mains()
elec.appliances



[Appliance(type='fridge', instance=1),
 Appliance(type='dish washer', instance=1),
 Appliance(type='electric water heating appliance', instance=1),
 Appliance(type='spin dryer', instance=1),
 Appliance(type='electric furnace', instance=1),
 Appliance(type='sockets', instance=1),
 Appliance(type='sockets', instance=2),
 Appliance(type='air conditioner', instance=1),
 Appliance(type='sockets', instance=3),
 Appliance(type='sockets', instance=4)]

In [32]:
split_point = elec.train_test_split(train_fraction=0.2)

In [33]:
split_point.date()

datetime.date(2014, 2, 24)

In [34]:
train = DataSet("/Users/nipunbatra/Downloads/wikienergy-2.h5")

In [35]:
train.set_window(end=split_point.date())



In [36]:
train.buildings[11].elec.plot()

In [12]:
test.set_window(start="30-4-2011")

Timestamp('2014-04-02 00:30:00-0500', tz='US/Central')

In [12]:
co = CombinatorialOptimisation()
co.train(elec)

Training model for submeter 'ElecMeter(instance=2, building=11, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])'
Training model for submeter 'ElecMeter(instance=3, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=1)])'
Training model for submeter 'ElecMeter(instance=4, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=2)])'
Training model for submeter 'ElecMeter(instance=5, building=11, dataset='WikiEnergy', appliances=[Appliance(type='dish washer', instance=1)])'
Training model for submeter 'ElecMeter(instance=6, building=11, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])'
Training model for submeter 'ElecMeter(instance=7, building=11, dataset='WikiEnergy', appliances=[Appliance(type='electric furnace', instance=1)])'
Training model for submeter 'ElecMeter(instance=8, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=3)])'
Tra

In [13]:
disag_filename = 'wikienergy-disag-co-new.h5'
output = HDFDataStore(disag_filename, 'w')
co.disaggregate(elec.mains(), output)
output.close()

vampire_power = 0.0 watts
Estimating power demand for 'ElecMeter(instance=2, building=11, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])'
Estimating power demand for 'ElecMeter(instance=3, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=1)])'
Estimating power demand for 'ElecMeter(instance=4, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=2)])'
Estimating power demand for 'ElecMeter(instance=5, building=11, dataset='WikiEnergy', appliances=[Appliance(type='dish washer', instance=1)])'
Estimating power demand for 'ElecMeter(instance=6, building=11, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])'
Estimating power demand for 'ElecMeter(instance=7, building=11, dataset='WikiEnergy', appliances=[Appliance(type='electric furnace', instance=1)])'
Estimating power demand for 'ElecMeter(instance=8, building=11, dataset='WikiEnergy', appliances=[Appliance(type='so

In [16]:
disag = DataSet(disag_filename)
disag_elec_co = disag.buildings[building_number].elec

In [17]:
pred_df_co = disag_elec['fridge'].load().next()[('power','active')]

In [18]:
fhmm = fhmm_exact.FHMM()
fhmm.train(elec)

Training model for submeter 'ElecMeter(instance=2, building=11, dataset='WikiEnergy', appliances=[Appliance(type='air conditioner', instance=1)])'
Training model for submeter 'ElecMeter(instance=3, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=1)])'
Training model for submeter 'ElecMeter(instance=4, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=2)])'
Training model for submeter 'ElecMeter(instance=5, building=11, dataset='WikiEnergy', appliances=[Appliance(type='dish washer', instance=1)])'
Training model for submeter 'ElecMeter(instance=6, building=11, dataset='WikiEnergy', appliances=[Appliance(type='spin dryer', instance=1)])'
Training model for submeter 'ElecMeter(instance=7, building=11, dataset='WikiEnergy', appliances=[Appliance(type='electric furnace', instance=1)])'
Training model for submeter 'ElecMeter(instance=8, building=11, dataset='WikiEnergy', appliances=[Appliance(type='sockets', instance=3)])'
Tra

In [19]:
disag_filename = 'wikienergy-disag-fhmm.h5'
output = HDFDataStore(disag_filename, 'w')
fhmm.disaggregate(elec.mains(), output)

KeyboardInterrupt: 

In [ ]:
disag = DataSet(disag_filename)
disag_elec = disag.buildings[building_number].elec 

In [ ]:
pred_df_fhmm = disag_elec['fridge'].load().next()[('power','active')]

In [20]:
from nilmtk.disaggregate.hart_85 import Hart85
h = Hart85()

In [21]:
h.train(elec.mains())

Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.


In [22]:
h.steady_states.head()

,active average
2014-02-01 00:21:00-06:00,305.000000
2014-02-01 00:29:00-06:00,637.000000
2014-02-01 00:33:00-06:00,456.939394
2014-02-01 01:07:00-06:00,308.125000
2014-02-01 01:15:00-06:00,452.444444


In [23]:
h.centroids

,"(power, active)"
0,152.154196
1,3401.577683
2,908.271235
3,5057.765816
4,1587.208333
5,8382.500000


In [24]:
disag_filename = 'wikienergy-disag-hart.h5'
output = HDFDataStore(disag_filename, 'w')
h.disaggregate(elec.mains(), output)
disag = DataSet(disag_filename)
disag_elec = disag.buildings[building_number].elec

Finding Edges, please wait ...
Edge detection complete.
Creating transition frame ...
Transition frame created.
Creating states frame ...
States frame created.
Finished.


In [27]:
ax1 = disag_elec['unknown', 0].load().next().plot()
pred_df_co.plot(ax=ax1, label="Combinatorial Optimisation")
elec['fridge', 1].load().next().plot(ax=ax1)
ax1.legend(["Predicted Hart", "Predicted CO","Ground truth"]);
plt.ylabel("Power (W)")
plt.xlabel("Time");